In [1]:
pip install ecpy

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pycryptodome

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install sympy

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install requests

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import math
import time
import random
import sympy
import warnings
from random import randint, seed
import sys
from ecpy.curves import Curve,Point
from Crypto.Hash import SHA3_256, HMAC, SHA256
import requests
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Util.Padding import pad
from Crypto.Util.Padding import unpad
import random
import re
import json

from random import randint, seed
from ecpy.curves import Curve
from Crypto.Hash import SHA3_256
from Crypto import Random   # a bit better secure random number generation 
import math

In [2]:
E = Curve.get_curve('secp256k1')
n = E.order
p = E.field
P = E.generator
a = E.a
b = E.b
print("Base point:\n", P)
print("p :", p)
print("a :", a)
print("b :", b)
print("n :", n)

k = Random.new().read(int(math.log(n,2)))
k = int.from_bytes(k, byteorder='big')%n

Q = k*P
print("\nQ:\n", Q)
print("Q on curve?", E.is_on_curve(Q))

Base point:
 (0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798 , 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8)
p : 115792089237316195423570985008687907853269984665640564039457584007908834671663
a : 0
b : 7
n : 115792089237316195423570985008687907852837564279074904382605163141518161494337

Q:
 (0x39ea310287e30f87a41e474a22c0555ba69a03152e18de04cb9c40f81d2a36c8 , 0x98adf20278f89f09e693a7e46cff07d3814cbac405f2ba7525de20dab71c9aa1)
Q on curve? True


In [3]:
import math
import time
import random
import sympy
import warnings
from random import randint, seed
import sys
from ecpy.curves import Curve,Point
from Crypto.Hash import SHA3_256, HMAC, SHA256
import requests
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Util.Padding import pad
from Crypto.Util.Padding import unpad
import random
import re
import json

API_URL = 'http://10.92.52.175:5000/'

stuID =  25139  # id of emre
stuID_ = 26846  # id of ekin
pseudo_cliend_ID = 18007 # id of the server


#server's Identitiy public key
IKey_Ser = Point(93223115898197558905062012489877327981787036929201444813217704012422483432813 , 8985629203225767185464920094198364255740987346743912071843303975587695337619, E)

def IKRegReq(h,s,x,y):
    mes = {'ID':stuID, 'H': h, 'S': s, 'IKPUB.X': x, 'IKPUB.Y': y}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "IKRegReq"), json = mes)		
    if((response.ok) == False): print(response.json())

def IKRegVerify(code):
    mes = {'ID':stuID, 'CODE': code}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "IKRegVerif"), json = mes)
    if((response.ok) == False): raise Exception(response.json())
    print(response.json())

def SPKReg(h,s,x,y):
    mes = {'ID':stuID, 'H': h, 'S': s, 'SPKPUB.X': x, 'SPKPUB.Y': y}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "SPKReg"), json = mes)		
    if((response.ok) == False): 
        print(response.json())
    else: 
        res = response.json()
        return res['SPKPUB.X'], res['SPKPUB.Y'], res['H'], res['S']

def OTKReg(id, keyID,x,y,hmac):
    mes = {'ID':id, 'KEYID': keyID, 'OTKI.X': x, 'OTKI.Y': y, 'HMACI': hmac}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "OTKReg"), json = mes)		
    print(response.json())
    if((response.ok) == False): return False
    else: return True


def ResetIK(rcode):
    mes = {'ID':stuID, 'RCODE': rcode}
    print("Sending message is: ", mes)
    response = requests.delete('{}/{}'.format(API_URL, "ResetIK"), json = mes)		
    print(response.json())
    if((response.ok) == False): return False
    else: return True

def ResetSPK(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.delete('{}/{}'.format(API_URL, "ResetSPK"), json = mes)		
    print(response.json())
    if((response.ok) == False): return False
    else: return True

def ResetOTK(id, h,s):
    mes = {'ID':id, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.delete('{}/{}'.format(API_URL, "ResetOTK"), json = mes)		
    print(response.json())

def PseudoSendMsgPH3(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "PseudoSendMsgPH3"), json = mes)		
    print(response.json())

def ReqMsg(id, h,s):
    mes = {'ID':id, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.get('{}/{}'.format(API_URL, "ReqMsg"), json = mes)	
    print(response.json())	
    if((response.ok) == True): 
        res = response.json()
        return res["IDB"], res["OTKID"], res["MSGID"], res["MSG"], res["EK.X"], res["EK.Y"]
    
def SendMsg(idA, idB, otkid, msgid, msg, ekx, eky):
    mes = {"IDA":idA, "IDB":idB, "OTKID": int(otkid), "MSGID": msgid, "MSG": msg, "EK.X": ekx, "EK.Y": eky}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "SendMSG"), json = mes)
    print(response.json())    
        
def reqOTKB(stuID, stuIDB, h, s):
    OTK_request_msg = {'IDA': stuID, 'IDB':stuIDB, 'S': s, 'H': h}
    print("Requesting party B's OTK ...")
    response = requests.get('{}/{}'.format(API_URL, "ReqOTK"), json = OTK_request_msg)
    print(response.json()) 
    if((response.ok) == True):
        print(response.json()) 
        res = response.json()
        return res['KEYID'], res['OTK.X'], res['OTK.Y']      
    else:
        return -1, 0, 0

def Status(stuID, h, s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.get('{}/{}'.format(API_URL, "Status"), json = mes)	
    print(response.json())
    if (response.ok == True):
        res = response.json()
        return res['numMSG'], res['numOTK'], res['StatusMSG']	



In [4]:
# IK data
sA = 75365099691681163466259435055151388598990196366045037773536006665067396329561
Q_x = 87058090705547232336742506266448902457032608537577399368869745072849975094263
Q_y = 73863361865121729253370141189521784002483174318560123943873444021233722393171

s_ik = 38435919104545028465405229083851199426577612833328471419512121279897034838259
h_ik = 40139398399024429188254655216662123008175195428486006903289784284292094948763

# SPK data
spk_private = 49760804016305672242628858452671561631883295017037579045439043666340119331411
spk_pub_x = 13279412479756039023006939294151267045839717479408817839293710814463378950452
spk_pub_y = 93617390663867776116902322986952603938589221775532373175859757173961357675363

h_spk = 70079813329073930856804606363140448062677695844504871159148958480169809247078
s_spk = 42702578452500864554658062321887469197282606023250231586097905168842648683915

# OTK data
otk_private = [82643481877244209249664745771237418234477690096430811959717657855485886045638, 41174952876732969873061234384234992214813764471036650196683240643681961363261, 5583145587846578787702511470815367992841936970155580384792772696588865599734, 10972659824334756040423367058252295538222729497376669524883637366213216276543, 24476832630378983018284218279520822890885956931179901813178437213634407485252, 97006687499945984175088601268948450846237282033793682470458998922800746644169, 74160389491792105526360889492360032639008866468092295253480592634938957625351, 2064776794757669561866729584104676040173421819183532961139782611452732109747, 58659695490434911403509798003920364225859338325533233728677532945968361081857, 19148444294741063747898074518353049744162854913178507691342149982373162093145]

In [5]:
def regenerationOTK():
    #numMsg, numOTK, StatusMsg = Status(stuID, h_ik, s_ik)
    #needed_OTK = 10 - numOTK
    #print(needed_OTK, "number of OTKs is needed.")
    ResetOTK(stuID, h_ik, s_ik)
    
    # OTKs generation
    hmac_array = [] 
    OTK_public = []
    OTK_private = []
    K_HMAC = b'\xc9\xc4\xb5\xb7|\xab8h:\x90\xc7\xcaf\xb5T\xe8\xffg\x1cSL\xb3NS\xc1\x18~-z\xbc\\"'

    for i in range(10): #needed_OTK
        print("for ID:", i)
        otk_private = random.randint(1, n-2) # random secret key
        otk_pub = otk_private * P # generating public key

        otk_pub_x = otk_pub.x
        otk_pub_y = otk_pub.y

        print("private:", otk_private)
        print("public:", otk_pub)
        print("public_x:", otk_pub_x)
        print("public_y:", otk_pub_y)

        otk_pub_x_byte_array = otk_pub_x.to_bytes((otk_pub_x.bit_length() +7)//8, byteorder = 'big')
        otk_pub_y_byte_array = otk_pub_y.to_bytes((otk_pub_y.bit_length() +7)//8, byteorder = 'big')
        otk_public_x_y = otk_pub_x_byte_array + otk_pub_y_byte_array #concatenation of x and y 

        hmac = HMAC.new(key = K_HMAC, msg = otk_public_x_y, digestmod = SHA256)
        
        print("hmac.hexdigest():", hmac.hexdigest())
        hmac_array.append(hmac)
        OTK_public.append(otk_pub)
        OTK_private.append(otk_private)

        OTKReg(stuID, i, otk_pub_x, otk_pub_y, hmac.hexdigest())
        print('------------------------------------------')
    return OTK_private

In [7]:
otk_private = regenerationOTK() # updating Emre's private OTKs

Sending message is:  {'ID': 25139, 'H': 40139398399024429188254655216662123008175195428486006903289784284292094948763, 'S': 38435919104545028465405229083851199426577612833328471419512121279897034838259}
All OTKs deleted !
for ID: 0
private: 103973071408047639745591123638003953321601097756999877384401407147503532101031
public: (0x6e7d35a59581317a6396cd3bd03a5cf2d91a1ff146519f3a92b969c9fc1a2a39 , 0x86af2c73cdcebdbaec49cef03ad72a0f6007e5b8b11e766bb9a4b22a09d51ef0)
public_x: 49975639483948859230800996960629959894396867134084774555646949347800932559417
public_y: 60919426745401106968214063757470094773803411368520771547333647198732569353968
hmac.hexdigest(): aafdf87eaa0ac5ffad4d4e07b328787fb34cf40d0a9c2dbe6abba88a158bc704
Sending message is:  {'ID': 25139, 'KEYID': 0, 'OTKI.X': 49975639483948859230800996960629959894396867134084774555646949347800932559417, 'OTKI.Y': 60919426745401106968214063757470094773803411368520771547333647198732569353968, 'HMACI': 'aafdf87eaa0ac5ffad4d4e07b328787fb34cf40d

In [8]:
numMsg, numOTK, StatusMsg = Status(stuID, h_ik, s_ik)

Sending message is:  {'ID': 25139, 'H': 40139398399024429188254655216662123008175195428486006903289784284292094948763, 'S': 38435919104545028465405229083851199426577612833328471419512121279897034838259}
{'numMSG': 0, 'numOTK': 10, 'StatusMSG': 'You have 0 unread messages in your mailbox.\n You have 10 OTKs left. The largest key id is 9'}


In [9]:
PseudoSendMsgPH3(h_ik, s_ik)

Sending message is:  {'ID': 25139, 'H': 40139398399024429188254655216662123008175195428486006903289784284292094948763, 'S': 38435919104545028465405229083851199426577612833328471419512121279897034838259}
Your favourite pseudo-client sent you 5 messages. You can get them from the server


In [10]:
# Requesting messages from the server:
otkID_array = []
msgID_array = []
msg_array = []
for i in range(5):
    stuIDB, otkID, msgID, msg, ek_x, ek_y = ReqMsg(stuID, h_ik, s_ik)
    msg_array.append(msg)
    otkID_array.append(otkID)
    msgID_array.append(msgID)

Sending message is:  {'ID': 25139, 'H': 40139398399024429188254655216662123008175195428486006903289784284292094948763, 'S': 38435919104545028465405229083851199426577612833328471419512121279897034838259}
{'IDB': 18007, 'OTKID': 0, 'MSGID': 1, 'MSG': 28538472581622255808811890334364106022880028416578151315468856361438572963321864534305366341433349449616939009035695265530434980666400483991280415650611604110763717400571871071451822827228671310944456, 'EK.X': 26794687761056960368204985523797658684904888173611643546352890030452822474873, 'EK.Y': 94790539885361241851666899773248842820712631694506840982675750602729228291583}
Sending message is:  {'ID': 25139, 'H': 40139398399024429188254655216662123008175195428486006903289784284292094948763, 'S': 38435919104545028465405229083851199426577612833328471419512121279897034838259}
{'IDB': 18007, 'OTKID': 0, 'MSGID': 2, 'MSG': 302072051086919593545216864560838783567243164857399728380292721739107636774644205492249303099910897051910000924050995133458279

In [11]:
# Getting messages from the Server:

first_iteration = True
decrypted_messages = []
counter = 0
for message in msg_array:
    message_byte_array = message.to_bytes((message.bit_length() +7)//8, byteorder = 'big')
    
    # hmac of the message
    message_HMAC = message_byte_array[len(message_byte_array)-32:]
    #print(message_HMAC) 

    # nonce and the ciphertext
    message_with_nonce = message_byte_array[:len(message_byte_array)-32]
    #print(message_with_nonce)

    # ciphertext
    ciphertext = message_byte_array[8:len(message_byte_array)-32]
    #print(ciphertext)


    # _session key generation start_
    EK_B_point = Point(ek_x, ek_y, curve = E)
    T = otk_private[otkID] * EK_B_point
    T_x = T.x
    T_y = T.y

    T_x_byte_array = T_x.to_bytes((T_x.bit_length() +7)//8, byteorder = 'big')
    T_y_byte_array = T_y.to_bytes((T_y.bit_length() +7)//8, byteorder = 'big')
    U = T_x_byte_array + T_y_byte_array + b'MadMadWorld'

    K_S = SHA3_256.new(U)
    # _session key generation end_

    # _key derivation start_
    if(first_iteration):
        K_KDF = K_S
        first_iteration = False
    else:
        K_KDF = SHA3_256.new(K_HMAC.digest() + b'YouWillNotHaveTheDrink')

    K_ENC = SHA3_256.new(K_KDF.digest() + b'LeaveMeAlone')
    K_HMAC = SHA3_256.new(K_ENC.digest() + b'GlovesAndSteeringWheel')
    # _key derivation end_

    # obtaining hmac from ciphertext
    hmac = HMAC.new(K_HMAC.digest(), ciphertext, digestmod=SHA256)
    hmac = hmac.digest()

    # checking MAC values
    if(hmac == message_HMAC):
        print("HMAC is verified")

        # dencryption of the message
        cipher = AES.new(K_ENC.digest(), AES.MODE_CTR, nonce = message_with_nonce[0:8])
        dtext = cipher.decrypt(message_with_nonce[8:])
        decrypted_message = dtext.decode('utf-8')

        print("Decrypted message:", decrypted_message)
        decrypted_messages.append(decrypted_message)

    else: 
        print("HMAC is not verified")


    counter = counter + 1


    print("_________________________________")
    

HMAC is verified
Decrypted message: https://www.youtube.com/watch?v=mJXUNMexT1c
_________________________________
HMAC is verified
Decrypted message: https://www.youtube.com/watch?v=KsEjdfXudfM
_________________________________
HMAC is verified
Decrypted message: https://www.youtube.com/watch?v=mJXUNMexT1c
_________________________________
HMAC is verified
Decrypted message: https://www.youtube.com/watch?v=mJXUNMexT1c
_________________________________
HMAC is verified
Decrypted message: https://www.youtube.com/watch?v=2aHkqB2-46k
_________________________________


In [12]:
def signID(ID):
    k = random.randint(1, n-2)
    R = k * P
    r = R.x % n
    
    r_byte_array = r.to_bytes((r.bit_length() +7)//8, byteorder = 'big')
    m_byte_array = ID.to_bytes((ID.bit_length() +7)//8, byteorder = 'big')
    r_m = r_byte_array + m_byte_array #concatenation of r and m where m is ID

    h = SHA3_256.new(r_m)
    h = int.from_bytes(h.digest(), byteorder='big') % n

    s = (k - (sA * h)) % n
    return s, h

In [13]:
# Signing psuedo client ID then getting the OTK information:
def getRecieverOTK(reciever_id):
    s_c, h_c = signID(reciever_id)
    key_id_B, otk_x_B, otk_y_B   = reqOTKB(stuID, reciever_id, h_c, s_c)
    return key_id_B, otk_x_B, otk_y_B

In [14]:
# Encrypting and Sending messages which we got from the server, to the SERVER:
key_id_B, otk_x_B, otk_y_B = getRecieverOTK(pseudo_cliend_ID)

first_iteration = True
counter = 0
encrypted_messages = []

for message in decrypted_messages:
    
    message_byte_array = bytes(message, 'utf-8')
    #    _session key generation start_

    publicOTK_B_point = Point(otk_x_B, otk_y_B, curve = E)

    private_EK = otk_private[key_id_B % 10] #random.randint(1, n-2) #otk_private[key_id_B] # random secret key
    public_EK = private_EK * P  # generating public key

    T = private_EK * publicOTK_B_point
    T_x = T.x
    T_y = T.y

    T_x_byte_array = T_x.to_bytes((T_x.bit_length() +7)//8, byteorder = 'big')
    T_y_byte_array = T_y.to_bytes((T_y.bit_length() +7)//8, byteorder = 'big')
    U = T_x_byte_array + T_y_byte_array + b'MadMadWorld'

    K_S = SHA3_256.new(U)
    # _session key generation end_    

    # _key derivation start_
    if(first_iteration):
        K_KDF = K_S
        first_iteration = False
    else:
        K_KDF = SHA3_256.new(K_HMAC.digest() + b'YouWillNotHaveTheDrink')

    K_ENC = SHA3_256.new(K_KDF.digest() + b'LeaveMeAlone')
    K_HMAC = SHA3_256.new(K_ENC.digest() + b'GlovesAndSteeringWheel')
    # _key derivation end_

    
    cipher = AES.new(K_ENC.digest(), AES.MODE_CTR)
    message = cipher.encrypt(message_byte_array)

    hmac = HMAC.new(K_HMAC.digest(), message, digestmod=SHA256)
    hmac = hmac.digest()

    ctext = cipher.nonce + message + hmac
    ctext = int.from_bytes(ctext, byteorder='big')
    encrypted_messages.append(ctext)

    print(ctext)

    SendMsg(stuID, pseudo_cliend_ID, key_id_B, counter, ctext, public_EK.x, public_EK.y)

    counter = counter + 1

    print("_________________________________")
    

Requesting party B's OTK ...
{'KEYID': 999, 'OTK.X': 30568881089037169074576707248814156158979475758222264003724744315000739721181, 'OTK.Y': 85993894123958406896957852312206763898726822265659964875343422367626452147261}
{'KEYID': 999, 'OTK.X': 30568881089037169074576707248814156158979475758222264003724744315000739721181, 'OTK.Y': 85993894123958406896957852312206763898726822265659964875343422367626452147261}
36545563832942636878632513062948229275930859174400035344993870986704203008294141064468161001042301460228815503962826083372370696707009708354982165476031701767129767146400635818654615412041653763993532
Sending message is:  {'IDA': 25139, 'IDB': 18007, 'OTKID': 999, 'MSGID': 0, 'MSG': 36545563832942636878632513062948229275930859174400035344993870986704203008294141064468161001042301460228815503962826083372370696707009708354982165476031701767129767146400635818654615412041653763993532, 'EK.X': 16871375104479339833186252357789905752851399453347817349062626167565133919010, 'EK.Y': 95377468

In [15]:
OTK_private_X = [] # private OTKs of Ekin
def resetReciever():
    # IK data (ekin)
    stuID = 26846
    sA = 26037317628054985808039282432726226222027022467068096985918786230345691957027
    Q_x = 34966638871069348443434840366429984004050324904300954986624560986102166574831
    Q_y = 18659130989987264139111259616350811871077628350857638087893010137535414968071

    s_ik = 10711401428822947968805605574831727095538167654217243597237904326250472141120
    h_ik = 2310173870477934111899724596823579375648537282076021725587322962459991319407

    # SPK data
    spk_private = 108276252172224876587334255735165101721975917778097283393900202750918785959710
    spk_pub_x = 97710353570137819193379937963704253670763857898842070763814037579480279837106
    spk_pub_y = 23341606178871725188404066641975812905580291605280961572149830211055662093589

    h_spk = 50104479202124469309008236680532873175969290556870197285989963025491089808550
    s_spk = 56354363435458804487927205429170920223354636346201640247065025929905617989041

    OTK_private = []

    ResetOTK(stuID, h_ik, s_ik)

    server_SPK_pub = (85040781858568445399879179922879835942032506645887434621361669108644661638219,
    46354559534391251764410704735456214670494836161052287022185178295305851364841,
    59007954485099251624961572416014350736776821380203722506041937308252698607740,
    101911760621645665479179352120241706761371492322672083429040706302642553897152)

    server_H = server_SPK_pub[2]
    server_S = server_SPK_pub[3]
    server_SPK_pub_x = server_SPK_pub[0]
    server_SPK_pub_y = server_SPK_pub[1]


    #Signature Verification:
    server_spk_pub_x_byte_array = server_SPK_pub_x.to_bytes((server_SPK_pub_x.bit_length() +7)//8, byteorder = 'big')
    server_spk_pub_y_byte_array = server_SPK_pub_y.to_bytes((server_SPK_pub_y.bit_length() +7)//8, byteorder = 'big')
    server_spk_x_y = server_spk_pub_x_byte_array + server_spk_pub_y_byte_array # concatenation of server_spk.x and server_spk.y 

    V = (server_S*P) + (server_H*IKey_Ser) 
    v = V.x % n
    v_byte_array = v.to_bytes((v.bit_length() +7)//8, byteorder = 'big')
    v_m = v_byte_array + server_spk_x_y
    h_ = SHA3_256.new(v_m)
    h_ = int.from_bytes(h_.digest(), byteorder='big') % n

    print("h_:", h_)
    print("h :", server_H)
    if(server_H == h_):
        print("verified")
    else:
        print("not verified")


    # Generating HMAC Key (KHMAC):
    server_SPK_pub_points = Point(server_SPK_pub_x, server_SPK_pub_y, curve = E)
    T = spk_private * server_SPK_pub_points
    T_x = T.x
    T_y = T.y

    T_x_byte_array = T_x.to_bytes((T_x.bit_length() +7)//8, byteorder = 'big')
    T_y_byte_array = T_y.to_bytes((T_y.bit_length() +7)//8, byteorder = 'big')
    joker = bytes("NoNeedToRideAndHide", 'utf-8')
    U = T_x_byte_array + T_y_byte_array + joker

    K_HMAC = SHA3_256.new(U)

    print("T_x_byte_array:", T_x_byte_array)
    print("T_y_byte_array:", T_y_byte_array)
    print("U:", U)
    print("K_HMAC:", K_HMAC.digest())
    
    # OTKs generation
    hmac_array = [] 
    OTK_public = []
    K_HMAC = K_HMAC.digest()

    for i in range(10): #needed_OTK
        print("for ID:", i)
        otk_private = random.randint(1, n-2) # random secret key
        otk_pub = otk_private * P # generating public key

        otk_pub_x = otk_pub.x
        otk_pub_y = otk_pub.y

        print("private:", otk_private)
        print("public:", otk_pub)
        print("public_x:", otk_pub_x)
        print("public_y:", otk_pub_y)

        otk_pub_x_byte_array = otk_pub_x.to_bytes((otk_pub_x.bit_length() +7)//8, byteorder = 'big')
        otk_pub_y_byte_array = otk_pub_y.to_bytes((otk_pub_y.bit_length() +7)//8, byteorder = 'big')
        otk_public_x_y = otk_pub_x_byte_array + otk_pub_y_byte_array #concatenation of x and y 

        hmac = HMAC.new(key = K_HMAC, msg = otk_public_x_y, digestmod = SHA256)
        
        print("hmac.hexdigest():", hmac.hexdigest())
        hmac_array.append(hmac)
        OTK_public.append(otk_pub)
        OTK_private_X.append(otk_private)

        OTKReg(stuID, i, otk_pub_x, otk_pub_y, hmac.hexdigest())
        print('------------------------------------------')




In [16]:
resetReciever()

Sending message is:  {'ID': 26846, 'H': 2310173870477934111899724596823579375648537282076021725587322962459991319407, 'S': 10711401428822947968805605574831727095538167654217243597237904326250472141120}
All OTKs deleted !
h_: 59007954485099251624961572416014350736776821380203722506041937308252698607740
h : 59007954485099251624961572416014350736776821380203722506041937308252698607740
verified
T_x_byte_array: b'\xd6\xbby\xd0\xe6\xc3\xe1\xd1\xef\xae\xfb/\x8f\x9a\x1b\x01P\x89\xd6z\x03\xa1\xcfc\xd6ZD/\x05\xe7\x1a\xac'
T_y_byte_array: b'\xb3\xa4\x01\xab\xc0\xce\xb5\xc24\x8dE\xd0jpw\x17\xebz\xab\xf1\xb0\xc9\xf2\x9a\x90\x94\x80\xb1\xb4\xee\xd6\xda'
U: b'\xd6\xbby\xd0\xe6\xc3\xe1\xd1\xef\xae\xfb/\x8f\x9a\x1b\x01P\x89\xd6z\x03\xa1\xcfc\xd6ZD/\x05\xe7\x1a\xac\xb3\xa4\x01\xab\xc0\xce\xb5\xc24\x8dE\xd0jpw\x17\xebz\xab\xf1\xb0\xc9\xf2\x9a\x90\x94\x80\xb1\xb4\xee\xd6\xdaNoNeedToRideAndHide'
K_HMAC: b"\xb3'\xf2\xda\xa7!I\x18\x8c\xbe\xdf\xa4\x12F3.\x1f/\xc6\xea1\x05\xa6\x80\xf9\xbe\x7fX\x14\xfd3,"
for I

In [17]:
# private OTKs of the Ekin
OTK_private_X

[60356411937189005082702512571900163863000898265996185139354719582833691012297,
 96048764973915882347836381015658423890267908828856715394933466706166213947479,
 25028276951242894110548106794336433827329705178652145529513560202903555882415,
 44829204912111856571907499737086946718867014472727444248637067098306949105198,
 20933153017289087180037917683836385568012365716285172460896776476605202828839,
 2305199324350114307330347355831553647644911991346599131147804601195647761490,
 91360978648451388783038066787987353359205009721990074185990306300599234697006,
 112721344879540877575008687108908951677680860779185833146045447844047409018565,
 37999641468674548018391390217485250509417634625658952070731653054421202530461,
 115505350453216214187672969934483104273456853874644752606678333323194425092963]

In [18]:
# Encrypting and Sending some messages to Ekin:
key_id_B, otk_x_B, otk_y_B = getRecieverOTK(stuID_) # recieving Ekin's OTK

first_iteration = True
counter = 0
encrypted_messages = []

#send_message = decrypted_messages
send_message = ["Merhaba Ekin", "Bakalim mesajlari alabilcen mi", "Naban", "kripto cok zevkli degil mi?", "Saygilar"]

for message in send_message:
    
    message_byte_array = bytes(message, 'utf-8')
    #    _session key generation start_

    publicOTK_B_point = Point(otk_x_B, otk_y_B, curve = E)

    private_EK = otk_private[key_id_B] #random.randint(1, n-2)#otk_private[key_id_B] # random secret key
    public_EK = private_EK * P  # generating public key

    T = private_EK * publicOTK_B_point
    T_x = T.x
    T_y = T.y

    T_x_byte_array = T_x.to_bytes((T_x.bit_length() +7)//8, byteorder = 'big')
    T_y_byte_array = T_y.to_bytes((T_y.bit_length() +7)//8, byteorder = 'big')
    U = T_x_byte_array + T_y_byte_array + b'MadMadWorld'

    K_S = SHA3_256.new(U)
    # _session key generation end_    

    # _key derivation start_
    if(first_iteration):
        K_KDF = K_S
        first_iteration = False
    else:
        K_KDF = SHA3_256.new(K_HMAC.digest() + b'YouWillNotHaveTheDrink')

    K_ENC = SHA3_256.new(K_KDF.digest() + b'LeaveMeAlone')
    K_HMAC = SHA3_256.new(K_ENC.digest() + b'GlovesAndSteeringWheel')
    # _key derivation end_

    
    cipher = AES.new(K_ENC.digest(), AES.MODE_CTR)
    message = cipher.encrypt(message_byte_array)

    hmac = HMAC.new(K_HMAC.digest(), message, digestmod=SHA256)
    hmac = hmac.digest()

    ctext = cipher.nonce + message + hmac
    ctext = int.from_bytes(ctext, byteorder='big')
    encrypted_messages.append(ctext)

    print(ctext)

    SendMsg(stuID, stuID_, key_id_B, counter, ctext, public_EK.x, public_EK.y)

    counter = counter + 1

    print("_________________________________")
    

Requesting party B's OTK ...
{'KEYID': 0, 'OTK.X': 33107979717006064075009906623231890176720433478204478639860561039501587197171, 'OTK.Y': 42359669420194052009733346935193772507203107809071042121429437940394554195233}
{'KEYID': 0, 'OTK.X': 33107979717006064075009906623231890176720433478204478639860561039501587197171, 'OTK.Y': 42359669420194052009733346935193772507203107809071042121429437940394554195233}
89444394837906589316172150538947398848555949135820057268255180051531809333187794121409492911749656049544696950233484999319493
Sending message is:  {'IDA': 25139, 'IDB': 26846, 'OTKID': 0, 'MSGID': 0, 'MSG': 89444394837906589316172150538947398848555949135820057268255180051531809333187794121409492911749656049544696950233484999319493, 'EK.X': 49975639483948859230800996960629959894396867134084774555646949347800932559417, 'EK.Y': 60919426745401106968214063757470094773803411368520771547333647198732569353968}
Your message sent succesfully
_________________________________
247972439990293319648

In [19]:
# This function is for showing that Ekin can recieve messages from emre
def toShowRecieving():
# IK data (ekin)
    stuID = 26846
    sA = 26037317628054985808039282432726226222027022467068096985918786230345691957027
    s_ik = 10711401428822947968805605574831727095538167654217243597237904326250472141120
    h_ik = 2310173870477934111899724596823579375648537282076021725587322962459991319407

    print("_________________________________")
    print("_________________________________")
    print("Status:")
    Status(stuID, h_ik, s_ik)
    print("_________________________________")
    print("_________________________________")

    # Signing Ekin's stuID with Ekin's private IK in order to get messages from the server:
    k = random.randint(1, n-2)
    R = k * P
    r = R.x % n

    r_byte_array = r.to_bytes((r.bit_length() +7)//8, byteorder = 'big')
    m_byte_array = stuID.to_bytes((stuID.bit_length() +7)//8, byteorder = 'big')
    r_m = r_byte_array + m_byte_array #concatenation of r and m where m is stuID

    h = SHA3_256.new(r_m)
    h = int.from_bytes(h.digest(), byteorder='big') % n

    s = (k - (sA * h)) % n
    print("s:", s)
    print("h:", h)

    # requesting messages from the server:
    otkID_array = []
    msgID_array = []
    msg_array = []
    for i in range(5):
        stuIDB, otkID, msgID, msg, ek_x, ek_y = ReqMsg(stuID, h,s)
        msg_array.append(msg)
        otkID_array.append(otkID)
        msgID_array.append(msgID)

    print(otkID)
    print("_________________________________")
    print("_________________________________")

    first_iteration = True
    counter = 0
    for message in msg_array:
        message_byte_array = message.to_bytes((message.bit_length() +7)//8, byteorder = 'big')
        
        # hmac of the message
        message_HMAC = message_byte_array[len(message_byte_array)-32:]
        #print(message_HMAC) 

        # nonce and the ciphertext
        message_with_nonce = message_byte_array[:len(message_byte_array)-32]
        #print(message_with_nonce)

        # ciphertext
        ciphertext = message_byte_array[8:len(message_byte_array)-32]
        #print(ciphertext)

        # _session key generation start_
        EK_B_point = Point(ek_x, ek_y, curve = E)
        T = OTK_private_X[otkID % 10] * EK_B_point
        T_x = T.x
        T_y = T.y

        T_x_byte_array = T_x.to_bytes((T_x.bit_length() +7)//8, byteorder = 'big')
        T_y_byte_array = T_y.to_bytes((T_y.bit_length() +7)//8, byteorder = 'big')
        U = T_x_byte_array + T_y_byte_array + b'MadMadWorld'


        K_S = SHA3_256.new(U)
        # _session key generation end_

        # _key derivation start_
        if(first_iteration):
            K_KDF = K_S
            first_iteration = False
        else:
            K_KDF = SHA3_256.new(K_HMAC.digest() + b'YouWillNotHaveTheDrink')

        K_ENC = SHA3_256.new(K_KDF.digest() + b'LeaveMeAlone')
        K_HMAC = SHA3_256.new(K_ENC.digest() + b'GlovesAndSteeringWheel')
        # _key derivation end_



        # obtaining hmac from ciphertext
        hmac = HMAC.new(K_HMAC.digest(), ciphertext, digestmod=SHA256)
        hmac = hmac.digest()
        
        # checking MAC values
        if(hmac == message_HMAC):
            print("HMAC is verified")

            # dencryption of the message
            cipher = AES.new(K_ENC.digest(), AES.MODE_CTR, nonce = message_with_nonce[0:8])
            dtext = cipher.decrypt(ciphertext)
            decrypted_message = dtext.decode('utf-8')

            print("Decrypted message:", decrypted_message)

        else: 
            print("HMAC is not verified")

        counter = counter + 1


        print("_________________________________")


    print("_________________________________")
    Status(stuID, h_ik, s_ik)
    
    

In [20]:
toShowRecieving()

_________________________________
_________________________________
Status:
Sending message is:  {'ID': 26846, 'H': 2310173870477934111899724596823579375648537282076021725587322962459991319407, 'S': 10711401428822947968805605574831727095538167654217243597237904326250472141120}
{'numMSG': 5, 'numOTK': 9, 'StatusMSG': 'You have 5 unread messages in your mailbox.\n You have 9 OTKs left. The largest key id is 9'}
_________________________________
_________________________________
s: 2461063999458575130939405692859156610230233811334708285024789777800818561457
h: 52819079444330054740796436410361966023677436896036343381561725376360455906597
Sending message is:  {'ID': 26846, 'H': 52819079444330054740796436410361966023677436896036343381561725376360455906597, 'S': 2461063999458575130939405692859156610230233811334708285024789777800818561457}
{'IDA': 25139, 'IDB': 26846, 'OTKID': 0, 'MSGID': 0, 'MSG': 894443948379065893161721505389473988485559491358200572682551800515318093331877941214094929117496